In [3]:
import data_preprocessing
import tests
import network
from keras.callbacks import ModelCheckpoint


In [3]:
model = network.create_model()

In [4]:
photo_names = data_preprocessing.load_photo_names_for_set('train')
descriptions = data_preprocessing.create_descriptions_dictionary_for_set(photo_names)
x1_train,x2_train,y_train = data_preprocessing.create_data_for_network(descriptions)

photo_names = data_preprocessing.load_photo_names_for_set('test')
descriptions = data_preprocessing.create_descriptions_dictionary_for_set(photo_names)
x1_test,x2_test,y_test = data_preprocessing.create_data_for_network(descriptions)

In [5]:
from keras.models import load_model
filepath = 'model/test.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

model = load_model(filepath)
checkpoint.best = model.evaluate([x1_test, x2_test], y_test)

n_epochs = 10

for i in range(n_epochs):
    print("Epoch %d" % i)
    model.fit([x1_train, x2_train], y_train, epochs = 5, verbose = 1, callbacks = [checkpoint], validation_data = ([x1_test, x2_test], y_test))
    model = load_model(filepath)


Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x000001F9FA721550>
Traceback (most recent call last):
  File "c:\users\katane\appdata\local\programs\python\python38\lib\weakref.py", line 346, in remove
    self = selfref()
KeyboardInterrupt: 


  1/352 [..............................] - ETA: 13:38 - loss: 4.9132

  4/352 [..............................] - ETA: 5s - loss: 4.2297   

  8/352 [..............................] - ETA: 5s - loss: 4.0941

 12/352 [>.............................] - ETA: 5s - loss: 3.9420

...

 98/352 [=======>......................] - ETA: 4s - loss: 3.7126

102/352 [=======>......................] - ETA: 4s - loss: 3.6853

106/352 [========>.....................] - ETA: 4s - loss: 3.6815

110/352 [========>.....................] - ETA: 3s - loss: 3.6997

KeyboardInterrupt: 

In [4]:
from keras.models import load_model

model = load_model('model/test.h5')
feature = data_preprocessing.extract_feature("external-content.duckduckgo.com.jpg")


In [29]:
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from numpy import array
x1 = []
x2 = []
x1.append(feature[0])
tokenizer = data_preprocessing.load_tokenizer()
tokenizer.word_index
current_sequence = [1]
current_sequence = pad_sequences([current_sequence], 34)[0]
x2.append(current_sequence)

y = model.predict((array(x1),array(x2)))
yhat = np.argmax(y)
word = word_for_id(yhat, tokenizer)
list(tokenizer.word_index)[12]



12

In [68]:
from numpy import argmax
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

def word_for_id(integer, tokenizer):
    tokenizer_list = list(tokenizer.word_index)
    if integer > len(tokenizer_list):
        return None
    else:
        return tokenizer_list[integer-1]


def generate_desc(model, tokenizer, feature, max_length):
	# seed the generation process
	in_text = 'startseq'
	# iterate over the whole length of the sequence
	for i in range(max_length):
		# integer encode input sequence
		sequence = tokenizer.texts_to_sequences([in_text])[0]
		# pad input
		sequence = pad_sequences([sequence], maxlen=max_length)
		# predict next word
		yhat = model.predict([feature,sequence], verbose=0)
		# convert probability to integer
		yhat = argmax(yhat)
		# map integer to word
		word = word_for_id(yhat, tokenizer)
		# stop if we cannot map the word
		if word is None:
			break
		# append as input for generating the next word
		in_text += ' ' + word
		# stop if we predict the end of the sequence
		if word == 'endseq':
			break
	return in_text

In [69]:
generate_desc(model, tokenizer, feature, 34)


'startseq two young boy be wear red shirt and be stand on the water endseq'

In [65]:
print(word_for_id(1, tokenizer))
print(list(tokenizer.word_index)[0])


startseq
startseq


dict_items([('startseq', 1), ('endseq', 2), ('in', 3), ('be', 4), ('on', 5), ('the', 6), ('dog', 7), ('and', 8), ('man', 9), ('with', 10), ('of', 11), ('two', 12), ('boy', 13), ('girl', 14), ('woman', 15), ('person', 16), ('white', 17), ('black', 18), ('run', 19), ('play', 20), ('wear', 21), ('stand', 22), ('to', 23), ('at', 24), ('jump', 25), ('water', 26), ('child', 27), ('young', 28), ('red', 29), ('an', 30), ('brown', 31), ('his', 32), ('sit', 33), ('blue', 34), ('through', 35), ('walk', 36), ('while', 37), ('hold', 38), ('shirt', 39), ('ball', 40), ('down', 41), ('little', 42), ('ride', 43), ('grass', 44), ('look', 45), ('snow', 46), ('over', 47), ('front', 48), ('three', 49), ('small', 50), ('field', 51), ('large', 52), ('up', 53), ('by', 54), ('green', 55), ('group', 56), ('one', 57), ('yellow', 58), ('her', 59), ('into', 60), ('air', 61), ('beach', 62), ('mouth', 63), ('near', 64), ('player', 65), ('rock', 66), ('dress', 67), ('bike', 68), ('street', 69), ('another', 70), ('for